In [ ]:
import numpy as np
import os, sys
import cv2
sys.path.append('cython_hw/')
import lfit_cython
from tutils import *
from bokeh.plotting import figure,output_file,show
from bokeh.layouts import gridplot

In [ ]:
#target_folder='/Users/Jaiyam/Dropbox/New Beads Videos/20190724 5min try3/'
target_folder='/Volumes/Users/Public/Sandhulab Data/Taisuke/20190809 control/'
all_files_for_analysis=extract_videos_for_processing(target_folder,extract_template=True)
#extract_temp_from_folder(target_folder+'pictures/')

In [ ]:
all_files_for_analysis=['/Volumes/Users/Public/Sandhulab Data/Taisuke/20190808 good/tracking/08082019_141300/1m/08082019_141300_1m.mov',
 '/Volumes/Users/Public/Sandhulab Data/Taisuke/20190808 good/tracking/08082019_141300/2m/08082019_141300_2m.mov',
 '/Volumes/Users/Public/Sandhulab Data/Taisuke/20190808 good/tracking/08082019_141300/3m/08082019_141300_3m.mov',
 '/Volumes/Users/Public/Sandhulab Data/Taisuke/20190808 good/tracking/08082019_142051/1m/08082019_142051_1m.mov',
 '/Volumes/Users/Public/Sandhulab Data/Taisuke/20190808 good/tracking/08082019_142051/2m/08082019_142051_2m.mov',
 '/Volumes/Users/Public/Sandhulab Data/Taisuke/20190808 good/tracking/08082019_142051/3m/08082019_142051_3m.mov']

In [ ]:
pp_list=[]
ffa=[]
for f in all_files_for_analysis:
    os.system("say 'Please select the dimensions of video'")
    fname=f #target_folder+f
    newname,pp_list=crop_and_trim(fname,pp_list)
    ffa.append(newname)
os.system("say 'Done'")

In [ ]:
#infile='/Users/Jaiyam/Dropbox/New Beads Videos/20190419 gravity/control.mov'
#outfile='/Users/Jaiyam/Dropbox/New Beads Videos/20190419 gravity/tracking/control/1m/control_1m.mov'
#crop_video(infile,outfile,46,56)
#extract_template_frames(all_files_for_analysis[1])

In [ ]:
for idx,fname in enumerate(ffa):
    template_file=target_folder+'temp1.jpg' #fname[:fname.rfind('/')+1]
    threshold=0.80
    try:
        track_video(fname,template_file,threshold)
        print('Video {}/{} finished tracking'.format(idx+1,len(all_files_for_analysis)))
    except Exception, e:
        print(str(e))
    print('\nDone')
os.system("say 'Done'")

# Post analysis of stopped beads with easy criterion

In [ ]:
ffa=['/Volumes/Users/Public/Sandhulab Data/Taisuke/20190808 good/tracking/08082019_141300/1m/tracking/08082019_141300_1m.mov',
 '/Volumes/Users/Public/Sandhulab Data/Taisuke/20190808 good/tracking/08082019_141300/2m/tracking/08082019_141300_2m.mov',
 '/Volumes/Users/Public/Sandhulab Data/Taisuke/20190808 good/tracking/08082019_141300/3m/tracking/08082019_141300_3m.mov',
 '/Volumes/Users/Public/Sandhulab Data/Taisuke/20190808 good/tracking/08082019_142051/1m/tracking/08082019_142051_1m.mov',
 '/Volumes/Users/Public/Sandhulab Data/Taisuke/20190808 good/tracking/08082019_142051/2m/tracking/08082019_142051_2m.mov',
 '/Volumes/Users/Public/Sandhulab Data/Taisuke/20190808 good/tracking/08082019_142051/3m/tracking/08082019_142051_3m.mov']

In [ ]:
target_folder='/Users/Jaiyam/Dropbox/New Beads Videos/20190318 ALL/exp1/'
fname=target_folder+'tracking/exp1_100nM/1m/exp1_100nM_1m.mov'
txtfile=fname[:fname.rfind('.')]+'_data.txt'
total_frames=240
bead_radius=3
tracked_objs=[]
tracked_objs=load_beads(txtfile)
last_frame_of_video=get_last_frame(fname)
highlighted_easy,num_stopped_easy,stopped_idxs_easy=highlight_stopped_beads(last_frame_of_video,tracked_objs,total_frames,bead_radius,std_threshold=1.0,strict=False)
cv2.imwrite(fname[:fname.rfind('.')+1]+'_{}_stopped_beads_easy.jpg'.format(num_stopped_easy),highlighted_easy)
stoppedtxt_easy=txtfile[:txtfile.rfind('.')]+'_stopped_easy.txt'
save_beads(stoppedtxt_easy,[tracked_objs[idx] for idx in stopped_idxs_easy])
print('Total beads tracked= {}'.format(len(tracked_objs)))
if num_stopped_easy>0:
    text2csv(stoppedtxt_easy)

## Post analysis of beads stopped in sensing area

In [ ]:
all_files_for_analysis=['/Users/Jaiyam/Dropbox/New Beads Videos/20190702 long/tracking/100nM/1m/100nM_1m.mov',
 '/Users/Jaiyam/Dropbox/New Beads Videos/20190702 long/tracking/100nM/2m/100nM_2m.mov',
 '/Users/Jaiyam/Dropbox/New Beads Videos/20190702 long/tracking/100nM/3m/100nM_3m.mov',
 '/Users/Jaiyam/Dropbox/New Beads Videos/20190702 long/tracking/100nM/4m/100nM_4m.mov',
 '/Users/Jaiyam/Dropbox/New Beads Videos/20190702 long/tracking/100nM/5m/100nM_5m.mov']

In [ ]:
#target_folder='/Users/Jaiyam/Dropbox/New Beads Videos/20190328 30mA 250G/'
#fname=target_folder+'tracking/control no NHS biotin on surface/1m/control no NHS biotin on surface_1m.mov'
oldres=None
failed=[]
succeeded=[]
for fname in all_files_for_analysis:
    txtfile=fname[:fname.rfind('.')]+'_data.txt'
    total_frames=240
    bead_radius=3
    tracked_objs=[]
    tracked_objs=load_beads(txtfile)
    debug=True
    try:
        highlighted_sa,num_stopped_sa,num_in_sa,oldres=find_beads_in_sensing_area(fname,tracked_objs,total_frames, bead_radius,strict=True,debug=debug,oldres=oldres)
        cv2.imwrite(fname[:fname.rfind('.')+1]+'_{}_stopped_beads_sa.jpg'.format(num_stopped_sa),highlighted_sa)
        stoppedtxt_sa=txtfile[:txtfile.rfind('.')]+'_stopped_sa.txt'
        #save_beads(stoppedtxt_sa,[tracked_objs[idx] for idx in stopped_idxs_sa])
        print('Total beads in sensing area= {}'.format(num_in_sa))
        with open(stoppedtxt_sa,'w') as f:
            msg='Number of beads in sensing area={}\n Number of beads stopped={}\n Percentage of beads stopped= {:.3f}\n'.\
            format(num_in_sa,num_stopped_sa,num_stopped_sa*100.0/num_in_sa)
            f.write(msg)
        succeeded.append(fname)
    except Exception, e:
        print('Error while analyzing file: {}'.format(fname))
        print(str(e))
        failed.append(fname)

    if len(succeeded)>0 and len(failed)>0:
        all_files_for_analysis.extend(failed) #modifying something while iterating over it. for shame!
        failed=[]

## Make graphs of bead paths

In [ ]:
target_folder='/Users/Jaiyam/Dropbox/New Beads Videos/20190710 tem/'
fname=target_folder+'tracking/20190705_121016-6ml/video.mov'
txtfile=fname[:fname.rfind('.')]+'_data.txt'
total_frames=35*15 #240
bead_radius=6 #3
tracked_objs=[]
tracked_objs=load_beads(txtfile)
good_beads=[62,4,25]#,14,47,2,3,59,25 #62,4
axis='y'
fps=15.0 #24.0
plot_pos_freq(tracked_objs,good_beads,fname[:fname.rfind('.')]+axis+'_pos.html',fs=fps,coord=axis)
last_frame=get_last_frame(fname)
cv2.imwrite(fname[:fname.rfind('.')]+'GBP.jpg',draw_full_paths_of_these_beads(last_frame,good_beads,tracked_objs))

In [ ]:
len(tracked_objs)

In [ ]:
last_frame_of_video=get_last_frame(fname)
highlighted,num_stopped,stopped_idxs=highlight_stopped_beads(last_frame_of_video,tracked_objs,total_frames,bead_radius,std_threshold=2.0)
cv2.imwrite(fname[:fname.rfind('.')+1]+'_{}_stopped_beads.jpg'.format(num_stopped),highlighted)
save_beads(stoppedtxt,[tracked_objs[idx] for idx in stopped_idxs])
if num_stopped>0:
    text2csv(stoppedtxt)
with open(fname[:fname.rfind('/')+1]+'correspondence.txt','w') as f:
    f.write(str([b for b in enumerate(stopped_idxs)]))

In [ ]:
last_frame_of_video=get_last_frame(fname)
highlighted,num_stopped,stopped_idxs=highlight_stopped_beads(last_frame_of_video,tracked_objs,total_frames,bead_radius,std_threshold=2.0,end=144)
cv2.imwrite(fname[:fname.rfind('.')+1]+'_{}_initial_stopped_beads.jpg'.format(num_stopped),highlighted)
# save_beads(stoppedtxt,[tracked_objs[idx] for idx in stopped_idxs])
# if num_stopped>0:
#     text2csv(stoppedtxt)
# with open(fname[:fname.rfind('/')+1]+'correspondence.txt','w') as f:
#     f.write(str([b for b in enumerate(stopped_idxs)]))

In [ ]:
len(tracked_objs)

In [ ]:
time_dict={idx: obj.tracked_frame_indices for idx,obj in enumerate(tracked_objs)}

In [ ]:
with open(fname[:fname.rfind('/')+1]+'frame_indices.txt','w') as f:
    f.write(str(time_dict))

In [ ]:
len(tracked_objs)

In [ ]:
len(tracked_objs[0].tracked_frame_indices)

In [ ]:
fname='/Users/Jaiyam/Dropbox/New Beads Videos/'

In [ ]:
p=figure()
print(sum([b.is_oscillating for b in tracked_objs]))
good_beads=[] #[264,342,810,828,218,336,646]
colors=['red','green','blue','black','orange','firebrick','fuchsia','indigo','magenta']
num=0
for idx,b in enumerate(tracked_objs):
    if b.is_oscillating:
        b.show_fft(p,'y',color=colors[num],display=False)
        num+=1
        good_beads.append(idx)
    else:
        pass
        #print('{} is not oscillating'.format(b))
    if num==9:
        break
    
show(p)

In [ ]:
blurred=cv2.GaussianBlur(frame,(5,5),3,3)

In [ ]:
cv2.imwrite('SAM/20181114/blurred.jpg',blurred)
cv2.imwrite('SAM/20181114/frame.jpg',frame)

In [ ]:
#tracked_objs=load_beads('16coil/withmag2_centers.txt')
#tracked_objs[509].num_frames_detected
#tracked_objs[509].is_oscillating

## 4k_external_trim_20_120 notes:
#### threshold=0.75
#### good beads are #s 166, 254, 13 (transition), [166,254,10,251,116,13]

## 4k_onchip_trim_70_170 notes:
#### threshold=0.75 template1
#### good beads are #s 57, 31, 210, 719,178 [57,31,234,719]

#### threshold=0.8 template2
#### good beads are #s 

## Planar coils notes (good beads):
#### 1Hz- 37,72,27;
#### 2 Hz- 52,123,132

## Onchip1um iPhone 4K notes

#### threshold=0.8
#### good beads are #s [325,433,84,545]

## SAM 1uM 0.5% notes

#### threshold=0.8
#### good beads are #s [1069,800]

## 16coil/nomag

#### threshold=0.8
#### good_beads=[49,59,65,27,25,261,173]

### Write a frame

In [ ]:
video=cv2.VideoCapture(fname)
count=0
ret=True
while ret:
    count+=ret
    ret,frame=video.read()
    if(count==1):
        cv2.imwrite(fname[:fname.rfind('/')+1]+'frame.jpg',frame)
        ret=False
video.release()

In [ ]:
draw_full_paths_of_these_beads(initial_frame, beads_ids, tracked_objs, color='green')

In [ ]:
folder="/Users/Jaiyam/Dropbox/New Beads Videos/20181203 SAM 15uM/tracking/expt3/"
clean_frame=get_last_frame(folder+"20181203_sam_15uM_3.mov")

yes=[443,186,44,152]

with open(folder+'20181203_sam_15uM_3_data.txt','r') as f:
    trajectories=eval(f.read())

tracked_objs=[]
for _ in range(len(trajectories)):
    tracked_objs.append(MovingObj((np.random.random(),np.random.random())))
    
for idx,obj in enumerate(tracked_objs):
    obj.previous_centers=trajectories[idx][:]
    
red_paths=draw_full_paths_of_these_beads(clean_frame,yes,tracked_objs,color='green')
#red_paths=draw_full_paths_of_these_beads(red_paths,nos,color='red')

In [ ]:
tracked_objs[0] is tracked_objs[1]

In [ ]:
cv2.imwrite(folder+'demo.jpg',red_paths)

In [ ]:
cv2.imwrite('16coil/paths_check.jpg',red_paths)

In [ ]:
cv2.imwrite('180nm_beads/last_frame.jpg',get_last_frame('180nm_beads/4coil_10x_mag_trim.mov'))

In [ ]:
#[obj.num_frames_detected for obj in tracked_objs]

In [ ]:
# struct moving_object
# centers in last num_frames_in_history frames
# is being tracked: boolean
# kalman filter to predict motion if object is being tracked

# maintain a list of all centers in previous num_frames_in_history frames
# for each object being tracked, predict the position in this frame
# for each center in this frame, check if the min distance to any predicted position is less than a threshold
# if true insert this center to the list of obj's centers in last few frames. update kalman filter
# if false, add to the list of objs not being tracked

In [ ]:
#[o.num_frames_detected for o in tracked_objs]

In [ ]:
[len(tracked_objs[idx].previous_centers) for idx in stopped_idxs]

In [ ]:
total_frames

In [ ]:
#good_beads=[166,254,10,251,116,13] <-- for external
#good_beads=[57,31,234,719]
#good_beads=[325,433,84,545] <-- for 1um beads
#good_beads=[1069,800]
#good_beads=[49,59,65,27,25,261,173] <-- 16coil/nomag
#good_beads=[243,303,41,229]

In [ ]:
std_threshold=1.0
xnoty=0
ynotx=0
xandy=0
for idx,obj in enumerate(tracked_objs):
    if len(obj.previous_centers)>=500:
        cen_x,cen_y=zip(*obj.previous_centers[-100:])
        if np.std(cen_x)<=std_threshold and np.std(cen_y)>std_threshold:
            #print('x but not y: ',idx,np.std(cen_x),np.std(cen_y))
            xnoty+=1
        elif np.std(cen_x)>std_threshold and np.std(cen_y)<=std_threshold:
            #print('y but not x: ',idx,np.std(cen_x),np.std(cen_y))
            ynotx+=1
        elif np.std(cen_x)<=std_threshold and np.std(cen_y)<=std_threshold:
            #print('Both x and y: ',idx,np.std(cen_x),np.std(cen_y))
            xandy+=1
print('xnoty: ',xnoty,', ynotx: ',ynotx,', xandy: ',xandy)

In [ ]:
cv2.imwrite('SAM/20181114/last_frame.jpg',last_frame)

In [ ]:
len(stopped_idxs)

In [ ]:
fname='/Users/Jaiyam/Dropbox/New Beads Videos/20181226 Biotin simple/tracking/1uM/20181226_biotin_1uM.mov'
txtfile=fname[:fname.rfind('/')+1]+'20181226_biotin_1uM_data.txt'
stoppedtxt=fname[:fname.rfind('/')+1]+'correspondence.txt'
with open(stoppedtxt,'r') as f:
    stopped_idxs=zip(*eval(f.read()))[1]

tracked_objs=load_beads(txtfile)
fps=24.0

In [ ]:
txtfile=fname[:fname.rfind('.')]+'_data.txt'
tracked_objs=load_beads(txtfile)

In [ ]:
good_beads=[12,19,23,24]#[2452,1426,2225]#,56]#,620]#56]
axis='y'
fps=24.0
plot_pos_freq(good_beads,fname[:fname.rfind('.')]+axis+'_pos.html',fs=fps,coord=axis)
last_frame=get_last_frame(fname)
cv2.imwrite(fname[:fname.rfind('.')]+'GBP.jpg',draw_full_paths_of_these_beads(last_frame,good_beads,tracked_objs))
#cv2.imwrite('good_beads_paths.jpg',writedistances(last_frame,[tracked_objs[b] for b in good_beads],float(count)/fps))

In [ ]:
stopped_ypositions=zip(*tracked_objs[19].previous_centers)[1]
stopped_ypositions=np.array([y-stopped_ypositions[0] for y in stopped_ypositions])
stopped_ypositions=np.gradient(stopped_ypositions)
brownian_ypositions=zip(*tracked_objs[23].previous_centers)[1]
brownian_ypositions=np.array([y-brownian_ypositions[0] for y in brownian_ypositions])
brownian_ypositions=np.gradient(brownian_ypositions)

In [ ]:
s_bins,s_edges=np.histogram(stopped_ypositions,bins=20)
b_bins,b_edges=np.histogram(brownian_ypositions,bins=20)

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
#fig=plt.figure()

In [ ]:
#ax=fig.subplots(1,1)
plt.plot(s_edges[:-1],s_bins,'b',b_edges[:-1],b_bins,'r')
#plt.plot()
plt.savefig('brownian_velocity.png')
plt.show()

In [ ]:
with open(fname[:fname.rfind('/')+1]+'correspondence.txt','w') as f:
    f.write(str([b for b in enumerate(stopped_idxs)]))

In [ ]:
with open('16coil/withmag2_centers.txt','w') as f:
    obj_dict={idx: obj.previous_centers for idx,obj in enumerate(tracked_objs)}
    f.write(str(obj_dict))

In [ ]:
plt.plot([c[0]/6.0 for c in tracked_objs[1].previous_centers]) #37,72,27,58,20,14
plt.show() #123,132

In [ ]:
def drawfft(array,fs=7.5):
    n=len(array)
    len_out=n//2 + 1
    maxf=fs/2.0 if n%2==0 else fs*(n-1)/(2.0*n)
    frequencies=maxf*np.arange(len_out)/len_out
    ys=np.abs(np.fft.rfft(array))
    #ys[1:]*=2
    plt.plot(frequencies,np.log10(ys))
    plt.xlabel='Frequency (Hz)'
    plt.ylabel='Amplitude (A.U.)'
    plt.show()
    return (frequencies,np.log10(ys))

In [ ]:
frequencies,amplitude=drawfft([c[0] for c in tracked_objs[0].previous_centers])

In [ ]:
import matplotlib.pyplot as plt
def make_histogram(objs,prop='num_frames_detected'):
    array=np.array([getattr(o,prop) for o in objs if getattr(o,prop)>=5])
    hist,edges=np.histogram(array,bins=20)
    plt.plot(edges[:-1],hist)
    plt.show()

In [ ]:
make_histogram(tracked_objs)

In [ ]:
tracker = cv2.TrackerMIL_create()

# Read video
video = cv2.VideoCapture("short copy (Converted).mov")

# Exit if video not opened.
if not video.isOpened():
    print "Could not open video"
    sys.exit()

# Read first frame.
ok, frame = video.read()
if not ok:
    print 'Cannot read video file'
    sys.exit()

# Define an initial bounding box
bbox = (287, 23, 86, 320)

# Uncomment the line below to select a different bounding box
bbox = cv2.selectROI(frame, False)

# Initialize tracker with first frame and bounding box
ok = tracker.init(frame, bbox)

while True:
    # Read a new frame
    ok, frame = video.read()
    if not ok:
        break

    # Update tracker
    ok, bbox = tracker.update(frame)

    # Draw bounding box
    if ok:
        p1 = (int(bbox[0]), int(bbox[1]))
        p2 = (int(bbox[0] + bbox[2]), int(bbox[1] + bbox[3]))
        cv2.rectangle(frame, p1, p2, (0,0,255))

    # Display result
    #cv2.imshow("Tracking", frame)
    videoWriter.write(frame)

    # Exit if ESC pressed
    k = cv2.waitKey(1) & 0xff
    if k == 27 : break

videoWriter.release()

In [ ]:
ret,frame=video.read()
gray=cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)
ret,thresh=cv2.threshold(gray,0,255,cv2.THRESH_BINARY+cv2.THRESH_OTSU)

for c in contours:
    #find bounding box co-ordinates
    (x,y),radius=cv2.minEnclosingCircle(c)
    #cast to integers
    center=(int(x),int(y))
    radius=int(radius)
    #draw the circle
    img=cv2.circle(frame,center,radius,(0,255,0),2)
    
    #find minimum area
    rect=cv2.minAreaRect(c)
    #calculate co-ordinates of minimum area rectangle
    box=cv2.boxPoints(rect)
    #normalize co-ordinates to integers
    box=np.int0(box)
    #cv2.drawContours(frame,[box],-1,(0,0,255),3)

#cv2.drawContours(frame,contours,-1,(255,0,0),1)
plt.imshow(frame)